In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import os
from scipy.sparse import csr_matrix, lil_matrix, eye, diags
from scipy.sparse.linalg import spsolve
from openpyxl import load_workbook
import requests
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from goatools.obo_parser import GODag
from collections import Counter


In [3]:
dir_name = os.path.join("..", "Data")

df_human = pd.read_csv(os.path.join(dir_name, "9606.protein.links.full.v12.0.txt"), sep=" ")
df_rvfv = pd.read_csv(os.path.join(dir_name, "string_interactions.tsv"), sep="\t")

df_denv = pd.read_csv(os.path.join(dir_name, "den_2_interactions.tsv"), sep="\t")

df_lasv = pd.read_csv(os.path.join(dir_name, "lassa_interactions.tsv"), sep="\t")

df_id_conversions = pd.read_csv(os.path.join(dir_name, "HUMAN_9606_idmapping_selected.tab"), sep="\t")
df_id_conversions.columns = ['Uniprot_ID', 'uk1', 'uk2', 'uk3', 'uk4', 'uk5', 'GO_IDs', 'uk6', 'uk7', 'uk8', 'uk9', 'uk10', 'uk11', 'uk12', 'uk13', 'uk14', 'uk15', 'uk16', 'ENSG', 'ENST', 'ENSP', 'uk17']

uniprot_ids = pd.read_csv(os.path.join(dir_name, "9606_uniprot_ids.tsv"), sep="\t")

godag = GODag(os.path.join(dir_name, "go.obo"))



C:\Users\socce\AppData\Local\Temp\ipykernel_22948\887215761.py:10: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_id_conversions = pd.read_csv(os.path.join(dir_name, "HUMAN_9606_idmapping_selected.tab"), sep="\t")


..\Data\go.obo: fmt(1.2) rel(2024-11-03) 43,983 Terms


In [6]:
uniprot_ids

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length
0,A0A024R1X5,unreviewed,A0A024R1X5_HUMAN,Beclin-1,BECN1 hCG_16958,Homo sapiens (Human),450
1,A0A024R274,unreviewed,A0A024R274_HUMAN,Mothers against decapentaplegic homolog (MAD h...,SMAD4,Homo sapiens (Human),552
2,A0A024R324,unreviewed,A0A024R324_HUMAN,Transforming protein RhoA,RHOA hCG_20136,Homo sapiens (Human),193
3,A0A024R6A3,unreviewed,A0A024R6A3_HUMAN,Presenilin (EC 3.4.23.-),PSEN1,Homo sapiens (Human),467
4,A0A024R7I7,unreviewed,A0A024R7I7_HUMAN,Ras-related protein Rab-3,NaN,Homo sapiens (Human),220
...,...,...,...,...,...,...,...
205099,X6RLT1,unreviewed,X6RLT1_HUMAN,Negative elongation factor complex member C/D,NELFCD,Homo sapiens (Human),593
205100,X6RLU5,unreviewed,X6RLU5_HUMAN,Calcium voltage-gated channel auxiliary subuni...,CACNA2D4,Homo sapiens (Human),192
205101,X6RLV5,unreviewed,X6RLV5_HUMAN,DEAD-box helicase 5,DDX5,Homo sapiens (Human),96
205102,X6RLY7,unreviewed,X6RLY7_HUMAN,Calcium voltage-gated channel auxiliary subuni...,CACNA2D4,Homo sapiens (Human),282


In [9]:
df_human

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
0,9606.ENSP00000000233,9606.ENSP00000356607,0,0,0,0,0,0,45,0,134,0,0,0,81,173
1,9606.ENSP00000000233,9606.ENSP00000427567,0,0,0,0,0,0,0,0,128,0,0,0,70,154
2,9606.ENSP00000000233,9606.ENSP00000253413,0,0,0,0,0,49,111,0,49,0,0,0,69,151
3,9606.ENSP00000000233,9606.ENSP00000493357,0,0,0,0,0,56,0,0,53,0,0,433,81,471
4,9606.ENSP00000000233,9606.ENSP00000324127,0,0,0,0,0,0,0,0,46,0,0,153,91,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,0,0,0,0,0,60,0,0,99,0,0,126,0,195
13715400,9606.ENSP00000501317,9606.ENSP00000370447,0,0,0,0,0,0,55,0,111,0,0,0,79,158
13715401,9606.ENSP00000501317,9606.ENSP00000312272,0,0,0,0,0,0,0,0,0,0,0,187,88,226
13715402,9606.ENSP00000501317,9606.ENSP00000402092,0,0,0,0,0,0,0,0,67,0,0,146,0,169


In [52]:
unique_prots_ensp = set(df_human["protein1"]) | set(df_human["protein2"])
unique_prots_ensp = [i.split('.')[1] for i in unique_prots_ensp]

In [53]:
ensp_id_list = list(df_id_conversions["ENSP"])
ensp_id_list = [str(i).split('; ') for i in ensp_id_list]

In [54]:
full_uniprot_id_list = list(df_id_conversions["Uniprot_ID"])

In [55]:
ensp_to_uniprot = {}

for keys, value in zip(ensp_id_list, full_uniprot_id_list):
    if isinstance(keys, list):
        for key in keys:  
            ensp_to_uniprot[key.split('.')[0]] = value
    else:
        ensp_to_uniprot[keys] = value  

In [85]:
unique_prots_uniprot = []
no_uniprot_id = []

for id in unique_prots_ensp:
    if id in ensp_to_uniprot:
        unique_prots_uniprot.append(ensp_to_uniprot[id])
    else:
        no_uniprot_id.append(id)
        #unique_prots_uniprot.append('null')


In [89]:
unique_prots_uniprot = set(unique_prots_uniprot)

In [105]:
len(unique_prots_uniprot)

19026

In [97]:
total_uniprot_id_list = list(uniprot_ids['Entry'])
total_gene_names = list(uniprot_ids['Gene Names'])

In [100]:
uniprot_to_gene_map = {u:g for u,g in zip(total_uniprot_id_list, total_gene_names)}

In [ ]:
unique_prots_genes = [uniprot_to_gene_map[p] for p in unique_prots_uniprot]

18834

In [75]:
prot1_int = list(df_human['protein1'])
prot2_int = list(df_human['protein2'])

In [82]:
new_prot1_int = []
new_prot2_int = []

for id1, id2 in zip(prot1_int, prot2_int):
    if id1 not in no_uniprot_id or id2 not in no_uniprot_id:
        new_prot1_int.append(id1.split('.')[1])
        new_prot2_int.append(id2.split('.')[1])
        

['ENSP00000356607',
 'ENSP00000427567',
 'ENSP00000253413',
 'ENSP00000493357',
 'ENSP00000324127',
 'ENSP00000325266',
 'ENSP00000320935',
 'ENSP00000371175',
 'ENSP00000480364',
 'ENSP00000388107',
 'ENSP00000461784',
 'ENSP00000381097',
 'ENSP00000234160',
 'ENSP00000241502',
 'ENSP00000317272',
 'ENSP00000487719',
 'ENSP00000354878',
 'ENSP00000479606',
 'ENSP00000369127',
 'ENSP00000493946',
 'ENSP00000383023',
 'ENSP00000266126',
 'ENSP00000262027',
 'ENSP00000271526',
 'ENSP00000364898',
 'ENSP00000325074',
 'ENSP00000310226',
 'ENSP00000006275',
 'ENSP00000369965',
 'ENSP00000258098',
 'ENSP00000265462',
 'ENSP00000347464',
 'ENSP00000330842',
 'ENSP00000358723',
 'ENSP00000363232',
 'ENSP00000498386',
 'ENSP00000225655',
 'ENSP00000351926',
 'ENSP00000447300',
 'ENSP00000264335',
 'ENSP00000482829',
 'ENSP00000387549',
 'ENSP00000386857',
 'ENSP00000271889',
 'ENSP00000377769',
 'ENSP00000489407',
 'ENSP00000341885',
 'ENSP00000307939',
 'ENSP00000297532',
 'ENSP00000222803',


In [46]:
ensp_to_uniprot

{'ENSP00000264335': 'P62258',
 'ENSP00000461762': 'P62258',
 'ENSP00000481059': 'P62258',
 'ENSP00000487356': 'P62258',
 'ENSP00000248975': 'Q04917',
 'ENSP00000306330': 'P61981',
 'ENSP00000340989': 'P31947',
 'ENSP00000238081': 'P27348',
 'ENSP00000371267': 'P27348',
 'ENSP00000309503': 'P63104',
 'ENSP00000379281': 'P63104',
 'ENSP00000379283': 'P63104',
 'ENSP00000379286': 'P63104',
 'ENSP00000379287': 'P63104',
 'ENSP00000379288': 'P63104',
 'ENSP00000395114': 'P63104',
 'ENSP00000398599': 'P63104',
 'ENSP00000430072': 'P63104',
 'ENSP00000263776': 'Q96QU6',
 'ENSP00000368109': 'Q4AC99',
 'ENSP00000261461': 'Q15172',
 'ENSP00000442866': 'Q15172',
 'ENSP00000164133': 'Q15173',
 'ENSP00000377669': 'Q14738',
 'ENSP00000420674': 'Q14738',
 'ENSP00000417963': 'Q14738',
 'ENSP00000337641': 'Q16537',
 'ENSP00000404632': 'Q16537',
 'ENSP00000452396': 'Q16537',
 'ENSP00000329009': 'Q13362',
 'ENSP00000333905': 'Q13362',
 'ENSP00000262239': 'Q13362',
 'ENSP00000412324': 'Q13362',
 'ENSP0000

In [11]:
df_lasv

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,combined_score
0,MYO5A,MYO5C,1858703,1845003,9606.ENSP00000382177,9606.ENSP00000261839,0,0,0,0.957,0.178,0.774,0,0.730,0.813
1,L,MYO5B,10004757,1846944,11620.L_LASSJ,9606.ENSP00000285039,0,0,0,0.000,0.000,0.000,0,0.603,0.603
2,L,MYO5C,10004757,1845003,11620.L_LASSJ,9606.ENSP00000261839,0,0,0,0.000,0.000,0.000,0,0.603,0.603
3,L,MYO5A,10004757,1858703,11620.L_LASSJ,9606.ENSP00000382177,0,0,0,0.000,0.000,0.000,0,0.603,0.603
4,DOCK7,DOCK6,1852363,1847494,9606.ENSP00000340742,9606.ENSP00000294618,0,0,0,0.978,0.000,0.596,0,0.410,0.600
5,L,DOCK6,10004757,1847494,11620.L_LASSJ,9606.ENSP00000294618,0,0,0,0.000,0.000,0.000,0,0.589,0.589
6,L,DOCK7,10004757,1852363,11620.L_LASSJ,9606.ENSP00000340742,0,0,0,0.000,0.000,0.000,0,0.589,0.589
7,L,MANBA,10004757,1843095,11620.L_LASSJ,9606.ENSP00000226578,0,0,0,0.000,0.000,0.000,0,0.579,0.579
8,L,PDE6B,10004757,1861473,11620.L_LASSJ,9606.ENSP00000420295,0,0,0,0.000,0.000,0.000,0,0.579,0.579
9,L,GUSB,10004757,1848445,11620.L_LASSJ,9606.ENSP00000302728,0,0,0,0.000,0.000,0.000,0,0.579,0.579


In [12]:
unique_prots = set(df_human["protein1"]) | set(df_human["protein2"])
#print(len(unique_prots))
unique_prots = list(unique_prots)
prot_map = {p:i for i,p in enumerate(unique_prots)}
prot_map


{'9606.ENSP00000430620': 0,
 '9606.ENSP00000387078': 1,
 '9606.ENSP00000236137': 2,
 '9606.ENSP00000419126': 3,
 '9606.ENSP00000290881': 4,
 '9606.ENSP00000363092': 5,
 '9606.ENSP00000485659': 6,
 '9606.ENSP00000483125': 7,
 '9606.ENSP00000305766': 8,
 '9606.ENSP00000362888': 9,
 '9606.ENSP00000356319': 10,
 '9606.ENSP00000352035': 11,
 '9606.ENSP00000361359': 12,
 '9606.ENSP00000434729': 13,
 '9606.ENSP00000497691': 14,
 '9606.ENSP00000366153': 15,
 '9606.ENSP00000341743': 16,
 '9606.ENSP00000222567': 17,
 '9606.ENSP00000340427': 18,
 '9606.ENSP00000474412': 19,
 '9606.ENSP00000256442': 20,
 '9606.ENSP00000493283': 21,
 '9606.ENSP00000265986': 22,
 '9606.ENSP00000258739': 23,
 '9606.ENSP00000475216': 24,
 '9606.ENSP00000371607': 25,
 '9606.ENSP00000323516': 26,
 '9606.ENSP00000415070': 27,
 '9606.ENSP00000005226': 28,
 '9606.ENSP00000388293': 29,
 '9606.ENSP00000222250': 30,
 '9606.ENSP00000297145': 31,
 '9606.ENSP00000264414': 32,
 '9606.ENSP00000485010': 33,
 '9606.ENSP00000365884':

In [13]:
# Generate list of unique host proteins that interact with NSs
rvfv_prot_list = set(pd.concat([df_rvfv["node1_external_id"], df_rvfv["node2_external_id"]], axis=0, ignore_index=True))
rvfv_interactor_list = {p for p in rvfv_prot_list if "9606.ENSP" in p}

s_array = np.zeros(len(unique_prots))
for prot in rvfv_interactor_list:
    if prot in prot_map:
        s_array[prot_map[prot]] = 1.0
    else:
        print(f'Protein {prot} does not interact with any human proteins.')

Protein 9606.ENSP00000356348 does not interact with any human proteins.
Protein 9606.ENSP00000384144 does not interact with any human proteins.
Protein 9606.ENSP00000360286 does not interact with any human proteins.
Protein 9606.ENSP00000353622 does not interact with any human proteins.


In [14]:
# For Dengue type 2

denv_prot_list = set(pd.concat([df_denv["node1_external_id"], df_denv["node2_external_id"]], axis=0, ignore_index=True))
denv_interactor_list = {p for p in denv_prot_list if "9606.ENSP" in p}

denv_s_array = np.zeros(len(unique_prots))
for prot in denv_interactor_list:
    if prot in prot_map:
        denv_s_array[prot_map[prot]] = 1.0
    else:
        print(f'Protein {prot} does not interact with any human proteins.')

Protein 9606.ENSP00000412788 does not interact with any human proteins.
Protein 9606.ENSP00000334499 does not interact with any human proteins.


In [15]:
# For Lassa virus

lasv_prot_list = set(pd.concat([df_lasv["node1_external_id"], df_lasv["node2_external_id"]], axis=0, ignore_index=True))
lasv_interactor_list = {p for p in lasv_prot_list if "9606.ENSP" in p}

lasv_s_array = np.zeros(len(unique_prots))
for prot in lasv_interactor_list:
    if prot in prot_map:
        lasv_s_array[prot_map[prot]] = 1.0
    else:
        print(f'Protein {prot} does not interact with any human proteins.')

Protein 9606.ENSP00000340742 does not interact with any human proteins.
Protein 9606.ENSP00000226578 does not interact with any human proteins.


In [16]:
# For RVFV from the IntAct database

int_prots = ['9606.ENSP00000264990','9606.ENSP00000299300','9606.ENSP00000295688','9606.ENSP00000377958','9606.ENSP00000280326','9606.ENSP00000286788',
    '9606.ENSP00000220764','9606.ENSP00000265651','9606.ENSP00000216281','9606.ENSP00000360709','9606.ENSP00000247207','9606.ENSP00000302961',
    '9606.ENSP00000296464','9606.ENSP00000324173','9606.ENSP00000318687','9606.ENSP00000480150','9606.ENSP00000316032','9606.ENSP00000360154',
    '9606.ENSP00000420813','9606.ENSP00000379182','9606.ENSP00000301071','9606.ENSP00000301072','9606.ENSP00000339001','9606.ENSP00000320295',
    '9606.ENSP00000264071','9606.ENSP00000341289'
 ]

ex_rvfv_s_array = np.zeros(len(unique_prots))

for prot in int_prots:
    if prot in prot_map:
        ex_rvfv_s_array[prot_map[prot]] = 1.0
    else:
        print(f'Protein {prot} does not interact with any human proteins.')




Protein 9606.ENSP00000216281 does not interact with any human proteins.
Protein 9606.ENSP00000360709 does not interact with any human proteins.
Protein 9606.ENSP00000247207 does not interact with any human proteins.
Protein 9606.ENSP00000318687 does not interact with any human proteins.
Protein 9606.ENSP00000420813 does not interact with any human proteins.
Protein 9606.ENSP00000379182 does not interact with any human proteins.
Protein 9606.ENSP00000301071 does not interact with any human proteins.
Protein 9606.ENSP00000301072 does not interact with any human proteins.


In [17]:
lasv_s_array.sum()


8.0

In [18]:
# Pair human protein columns, map protein names to matrix coordinates, change matrix value to one at that position (and inverse position)
w_sparse_matrix = lil_matrix((len(prot_map), len(prot_map)))

for prot1, prot2 in zip(df_human['protein1'], df_human['protein2']):
    w_sparse_matrix[prot_map[prot1], prot_map[prot2]] = 1
    w_sparse_matrix[prot_map[prot2], prot_map[prot1]] = 1

w_sparse_matrix = w_sparse_matrix.tocsr()




In [19]:
# Build a function that trims the network to a certain number of rounds of connection
def select_network_subset(starting_nodes, full_network, iterations, s_array=None):
    if s_array is None:
        s_array = starting_nodes
    
    # Build out network for given number of cycles
    if iterations > 0:
        new_starting_nodes = (full_network@starting_nodes) + starting_nodes
        return select_network_subset(new_starting_nodes, full_network, iterations - 1, s_array=s_array)
    
    else:
        starting_nodes[starting_nodes > 0.0] = 1.0
        print(f"Starting nodes: {starting_nodes.sum()}")
        
        trimmed_network_list = [p for p,m in zip(unique_prots, starting_nodes) if m > 0]

        starting_mask = starting_nodes > 0

        trimmed_network = full_network[starting_mask,:][:,starting_mask]

        trimmed_s_array = s_array[starting_mask]
        

        return trimmed_s_array, trimmed_network, trimmed_network_list


        

In [20]:
# Run algorithm 

def calculate_scores(s_array, network, network_list, d_or_i='D', alpha=0.5):
    D = np.sqrt(network.sum(-1))
    D[D < 1] = 1

    normalized_matrix = network.multiply(1.0/D)
    normalized_matrix = normalized_matrix.multiply(1.0/D.T)

    I = eye(len(network_list))

    if d_or_i.lower() == 'd':
        D_tilde = normalized_matrix.sum(axis=1).A1
        #D_tilde = csr_matrix(np.matrix(np.diag(D_tilde)))
        D_tilde = diags(D_tilde, format='csr')

        L = D_tilde - normalized_matrix

    elif d_or_i.lower() == 'i':
        L = I - normalized_matrix

    else:
        print(f'{d_or_i} is an improper input for Laplacian calculation. Select D or I.')

    M = I + (alpha * L)
    y_array = spsolve(M, s_array)

    return y_array


In [21]:
# Pair scores with protein names and filter

def sort_filter_scores(y_array, network_list, viral_interactor_list, filter=True):
    prots_scores_zipped = list(zip(y_array.tolist(), network_list))
    prots_scores_sorted = sorted(prots_scores_zipped, key=lambda x: x[0], reverse=True)
    
    if filter == True:
        prots_scores_final = [tup for tup in prots_scores_sorted if tup[1] not in viral_interactor_list]
    else:
        prots_scores_final = prots_scores_sorted

    return prots_scores_final

In [22]:
# I and one-step network RVFV

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 1)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='I', alpha=0.5)

prot_scores_I_one = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 7837.0


In [23]:
# D tilde and one-step network RVFV

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 1)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_D_one = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list, filter=False)

Starting nodes: 7837.0


In [21]:
# I and full network RVFV

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='I', alpha=0.5)

prot_scores_I_full = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 19622.0


In [17]:
# D tilde and full network RVFV

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_D_full = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list, filter=False)

Starting nodes: 19622.0


In [18]:
# D tilde and full network with Dengue type 2

s_array_one, network_one, network_list_one = select_network_subset(denv_s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_denv = sort_filter_scores(y_array_one, network_list_one, denv_interactor_list, filter=False)

Starting nodes: 19622.0


In [453]:
# D tilde and full network with Lassa virus

s_array_one, network_one, network_list_one = select_network_subset(lasv_s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_lasv = sort_filter_scores(y_array_one, network_list_one, lasv_interactor_list, filter=False)

Starting nodes: 19622.0


MemoryError: Unable to allocate 2.87 GiB for an array with shape (19622, 19622) and data type float64

In [513]:
# D tilde and full network with expanded RVFV from IntAct DB

s_array_one, network_one, network_list_one = select_network_subset(ex_rvfv_s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_ex_rvfv = sort_filter_scores(y_array_one, network_list_one, ex_rvfv_s_array, filter=False)

Starting nodes: 19622.0


In [515]:
# Add scores to dataframe

scores_df = pd.DataFrame(index=[p.split('.')[1] for _,p in prot_scores_D_full], columns=["RVFV"])

scores = [
    (prot_scores_D_full, 'RVFV'),
    (prot_scores_denv, 'DENV'),
    (prot_scores_lasv, 'LASV'),
    (prot_scores_ex_rvfv, 'RVFVex')
]

for prot_scores, column_name in scores:
    for s, p in prot_scores:
        scores_df.loc[p.split('.')[1], column_name] = s

In [516]:
scores_df

,RVFV,DENV,LASV,RVFVex
ENSP00000265651,0.714409,2.489288e-04,3.965504e-05,0.714192
ENSP00000360154,0.659057,4.618905e-05,3.075052e-04,0.659134
ENSP00000451560,0.648018,4.566916e-04,3.546594e-04,0.000815
ENSP00000369581,0.647397,6.473888e-01,3.045371e-04,0.000545
ENSP00000377958,0.642018,5.445292e-04,5.433123e-05,0.642814
...,...,...,...,...
ENSP00000485141,0.000001,9.474587e-07,5.989195e-07,0.000002
ENSP00000497579,0.000001,7.816776e-07,7.377077e-07,0.000002
ENSP00000499042,0.000001,8.156735e-07,1.143150e-06,0.000002
ENSP00000497547,0.000001,8.322699e-07,7.347714e-07,0.000002


In [24]:
df_id_conversions

,Uniprot_ID,uk1,uk2,uk3,uk4,uk5,GO_IDs,uk6,uk7,uk8,...,uk11,uk12,uk13,uk14,uk15,uk16,ENSG,ENST,ENSP,uk17
0,P62258,1433E_HUMAN,7531,NP_006752.1,374074368; 62131678; 67464424; 194379794; 2210...,2BR9:A; 3UAL:A; 3UBW:A; 6EIH:A; 7C8E:A; 7C8E:B...,GO:0005737; GO:0005829; GO:0005783; GO:0070062...,UniRef100_P62258,UniRef90_P62258,UniRef50_P62258,...,9606,605066,NaN,7644510; 8858348; 8684458; 20417184; 14702039;...,U20972; U54778; U43399; U43430; U28936; AB0171...,AAC50175.1; AAC50710.1; AAC50625.1; AAD00026.1...,ENSG00000108953.17; ENSG00000274474.3,ENST00000264335.13; ENST00000571732.5; ENST000...,ENSP00000264335.8; ENSP00000461762.1; ENSP0000...,15838597; 11782387; 12657644; 14966136; 153649...
1,Q04917,1433F_HUMAN,7533,NP_003396.1,4507951; 460779; 437363; 83754686; 12804373; 8...,2C63:A; 2C63:B; 2C63:C; 2C63:D; 2C74:A; 2C74:B...,GO:0150048; GO:0005737; GO:0005829; GO:0070062...,UniRef100_Q04917,UniRef90_Q04917,UniRef50_Q04917,...,9606,113508,NaN,8218406; 1578511; 8561965; 8812417; 15461802; ...,L20422; X80536; X78138; X57345; D78577; S80794...,AAA35483.1; CAA56676.1; CAA55017.1; CAA40620.1...,ENSG00000128245.15,ENST00000248975.6,ENSP00000248975.5,10206237; 11121172; 11996670; 12176995; 124801...
2,P61981,1433G_HUMAN,7532,NP_036611.2,6016838; 380764684; 635576381; 82407956; 82407...,2B05:A; 2B05:B; 2B05:C; 2B05:D; 2B05:E; 2B05:F...,GO:0005737; GO:0005829; GO:0070062; GO:0005925...,UniRef100_P61981,UniRef90_P61981,UniRef50_P61981,...,9606,605356; 617665,NaN,10433554; 10486217; 12853948; 15489334; 235725...,AF142498; AB024334; CR541904; CR541925; AC0063...,AAD48408.1; BAA85184.1; CAG46702.1; CAG46723.1...,ENSG00000170027.7,ENST00000307630.5,ENSP00000306330.3,11824616; 11996670; 12364343; 12482592; 150572...
3,P31947,1433S_HUMAN,2810,NP_006133.1,436408756; 306991738; 969812714; 350610438; 35...,1YWT:A; 1YWT:B; 1YZ5:A; 1YZ5:B; 3IQJ:A; 3IQU:A...,GO:0005737; GO:0005829; GO:0070062; GO:0005615...,UniRef100_P31947,UniRef90_P31947,UniRef50_P31947,...,9606,601290,NaN,1390337; 8515476; 9659898; 16710414; 15489334;...,M93010; X57348; AF029081; AF029082; CR541905; ...,AAA59546.1; CAA40623.1; AAC52029.1; AAC52030.1...,ENSG00000175793.12,ENST00000339276.6,ENSP00000340989.4,10969776; 12582028; 12730237; 12787309; 145172...
4,P27348,1433T_HUMAN,10971,NP_006817.1,71042777; 34785143; 32464; 3387922; 119621382;...,2BTP:A; 2BTP:B; 5IQP:A; 5IQP:B; 6BCR:A; 6BCR:B...,GO:0005737; GO:0005829; GO:0070062; GO:0005925...,UniRef100_P27348,UniRef90_P27348,UniRef50_P27348,...,9606,609009,NaN,2015305; 8515476; 15489334; 12665801; 23572552...,X56468; X57347; BT020014; CH471053; CH471053; ...,CAA39840.1; CAA40622.1; AAV38817.1; EAX00977.1...,ENSG00000134308.14,ENST00000238081.8; ENST00000381844.8,ENSP00000238081.3; ENSP00000371267.4,12535517; 15196694; 15930726; 16176975; 162154...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205009,Q9NSA8,Q9NSA8_HUMAN,NaN,NaN,8217330,NaN,NaN,UniRef100_Q9NSA8,UniRef90_Q9NSA8,UniRef50_Q9NSA8,...,9606,NaN,NaN,NaN,Z46940,CAB92527.1,NaN,NaN,NaN,NaN
205010,B7Z3X3,B7Z3X3_HUMAN,1837,NP_001185872.1,221041364; 312147278,NaN,GO:0008270,UniRef100_M0QZ28,UniRef90_Q9Y4J8-15,UniRef50_Q9Y4J8-15,...,9606,NaN,NaN,NaN,AK296450,BAH12359.1,NaN,NaN,NaN,12416719; 12475945; 12604589; 12899872; 158346...
205011,Q3YL84,Q3YL84_HUMAN,NaN,NaN,73544660,NaN,NaN,UniRef100_Q3YL84,UniRef90_Q3YL84,UniRef50_Q3YL84,...,9606,NaN,NaN,16110330,DQ102772,AAZ78118.1,NaN,NaN,NaN,19116388; 11877378; 12110597; 12170762; 123518...
205012,Q53S20,Q53S20_HUMAN,NaN,NaN,62988762,NaN,NaN,UniRef100_Q53S20,UniRef90_Q53S20,UniRef50_Q53S20,...,9606,NaN,NaN,NaN,AC073476,AAY24149.1,NaN,NaN,NaN,NaN


In [25]:
df_id_conversions.columns

Index(['Uniprot_ID', 'uk1', 'uk2', 'uk3', 'uk4', 'uk5', 'GO_IDs', 'uk6', 'uk7',
       'uk8', 'uk9', 'uk10', 'uk11', 'uk12', 'uk13', 'uk14', 'uk15', 'uk16',
       'ENSG', 'ENST', 'ENSP', 'uk17'],
      dtype='object')

In [26]:
# Create list of ENSP IDs

ensp_id_list = list(df_id_conversions["ENSP"])
ensp_id_list = [str(i).split('; ') for i in ensp_id_list]

In [27]:
# Create list of Uniprot IDs

uniprot_id_list = list(df_id_conversions["Uniprot_ID"])

NameError: name 'uniprot_id_list' is not defined

In [28]:
# Create ENSP to Uniprot ID conversion dictionary

ensp_to_uniprot = {}

for keys, value in zip(ensp_id_list, uniprot_id_list):
    if isinstance(keys, list):
        for key in keys:  
            ensp_to_uniprot[key.split('.')[0]] = value
    else:
        ensp_to_uniprot[keys] = value  

In [29]:
# Check for outdated ENSP IDs not included in the Uniprot database

not_in_database = []

for id in scores_df.index:
    if id not in ensp_to_uniprot:
        not_in_database.append(id)

NameError: name 'scores_df' is not defined

In [30]:
# Drop outdated ENSP IDs from our results

scores_df = scores_df.drop(not_in_database)    

NameError: name 'scores_df' is not defined

In [31]:
# Add rank columns for each run

rank_columns = ['RVFV', 'DENV', 'LASV', 'RVFVex']

for col in rank_columns:
    scores_df = scores_df.sort_values(col, ascending=False)
    scores_df[f'{col}_rank'] = list(range(1, len(scores_df) + 1))

NameError: name 'scores_df' is not defined

In [32]:
scores_df = scores_df.sort_values('RVFVex', ascending=False)

NameError: name 'scores_df' is not defined

In [33]:
# Convert dataframe index from ENSP to Uniprot IDs

scores_df.rename(index=ensp_to_uniprot, inplace=True)


NameError: name 'scores_df' is not defined

In [34]:
# Create list of GO IDs for each for each Uniprot ID

go_id_list = list(df_id_conversions["GO_IDs"])
go_id_list = [str(i).split('; ') for i in go_id_list]

In [35]:
# Map GO to Uniprot IDs in a dictionary

go_to_uniprot = {}

for value, key in zip(go_id_list, uniprot_id_list):
    go_to_uniprot[key] = value  

In [527]:
# Create list of GO IDs in the same order as the dataframe 

go_list = []
for i in scores_df.index:
    go_list.append(go_to_uniprot[i])

In [528]:
scores_df['GO_IDs'] = go_list

In [529]:
# Translate GO IDs to GO terms in a new list

go_term_list = [row[:] for row in go_list]
exclude_list = []

for i in range(len(go_list)):
    for j in range(len(go_list[i])):
        try:
            go_term_list[i][j] = godag[go_list[i][j]].name
        except KeyError:
            exclude_list.append(go_list[i][j])
            go_term_list[i][j] = f'GO ID {go_list[i][j]} not converted'

In [530]:
scores_df['GO_Terms'] = go_term_list

In [531]:
scores_df = scores_df.sort_values('RVFVex', ascending=False)

In [534]:

#scores_df[['RVFV', 'GO_Terms']][:15]
scores_df[:15]

,RVFV,DENV,LASV,RVFVex,RVFV_rank,DENV_rank,LASV_rank,RVFVex_rank,GO_IDs,GO_Terms
Q9UK99,0.714409,0.000249,0.000040,0.714192,1,4187,10072,1,"[GO:0005737, GO:0005829, GO:0005654, GO:001900...","[cytoplasm, cytosol, nucleoplasm, SCF ubiquiti..."
Q709F0,0.000032,0.000037,0.000690,0.689113,10821,11946,503,2,"[GO:0005737, GO:0005743, GO:0031966, GO:000573...","[cytoplasm, mitochondrial inner membrane, mito..."
Q01968,0.659057,0.000046,0.000308,0.659134,2,9130,2836,3,"[GO:0005905, GO:0030136, GO:0005737, GO:000582...","[clathrin-coated pit, clathrin-coated vesicle,..."
P50990,0.000195,0.000682,0.000048,0.658148,4047,351,6910,4,"[GO:0035578, GO:0044297, GO:0005813, GO:000583...","[azurophil granule lumen, cell body, centrosom..."
P52948,0.000173,0.000350,0.000046,0.647233,4758,2444,7571,5,"[GO:0005829, GO:0016604, GO:0005635, GO:004240...","[cytosol, nuclear body, nuclear envelope, nucl..."
O95757,0.000252,0.000179,0.000132,0.646038,2439,5949,5024,6,"[GO:0005737, GO:0005829, GO:0005634, GO:000077...","[cytoplasm, cytosol, nucleus, adenyl-nucleotid..."
Q9Y4L1,0.000247,0.000063,0.000497,0.644889,2555,6901,1406,7,"[GO:0071682, GO:0005783, GO:0034663, GO:000578...","[endocytic vesicle lumen, endoplasmic reticulu..."
P50991,0.642018,0.000545,0.000054,0.642814,5,783,5905,8,"[GO:0044297, GO:0042995, GO:0005813, GO:000583...","[cell body, cell projection, centrosome, chape..."
P49368,0.000244,0.000531,0.000059,0.639154,2648,861,5563,9,"[GO:0044297, GO:0005832, GO:0005856, GO:000582...","[cell body, chaperonin-containing T-complex, c..."
P48643,0.000234,0.000502,0.000056,0.633884,2867,1010,5765,10,"[GO:0044297, GO:0005813, GO:0005832, GO:000582...","[cell body, centrosome, chaperonin-containing ..."


In [396]:
new_df = scores_df[['LASV', 'GO_Terms']]

In [398]:
results_sheet = "C:\\Users\\socce\\OneDrive\\Documents\\Kehn-Hall Lab Data\\Python\\viral_interact\\Data\\network_prop_results.xlsx"

with pd.ExcelWriter(results_sheet, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    new_df.to_excel(writer, sheet_name='Sheet36', index=True)